# Carga de datos

In [2]:
!pip install unidecode
!pip install nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')


[nltk_data] Downloading package stopwords to /home/sergon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/sergon/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/sergon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# Librerías para manejo de datos
import numpy as np
np.random.seed(3301)
import pickle
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
pd.set_option('display.max_columns', 25) # Número máximo de columnas a mostrar
pd.set_option('display.max_rows', 50) # Numero máximo de filas a mostar
import pandas as pd
# Para preparar los datos
from sklearn.preprocessing import LabelEncoder
# Para crear el arbol de decisión
from sklearn.tree import DecisionTreeClassifier
# Para usar KNN como clasificador
from sklearn.neighbors import KNeighborsClassifier
# Para realizar la separación del conjunto de aprendizaje en entrenamiento y test.
from sklearn.model_selection import train_test_split
# Para evaluar el modelo
#from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score, accuracy_score
#from sklearn.metrics import plot_confusion_matrix
# Para búsqueda de hiperparámetros
from sklearn.model_selection import GridSearchCV
# Para la validación cruzada
from sklearn.model_selection import KFold
#Librerías para la visualización
import matplotlib.pyplot as plt
# Seaborn
import seaborn as sns
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.feature_extraction import text
from nltk.corpus import stopwords
from wordcloud import WordCloud
import unidecode
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import spacy
from sklearn.pipeline import Pipeline
#import joblib
#from sklearn.base import BaseEstimator, TransformerMixin

In [4]:
ubi = './data/ODScat_345.xlsx'

In [5]:
df_ods = pd.read_excel(ubi)

In [6]:
df_ods.shape

(4049, 2)

Se evidencion que hay 4049 opiniones sobre problematicas relacionadas con los ODS

In [8]:
df_ods.sample(5)

,Textos_espanol,sdg
3202,"Por lo tanto, necesitan desarrollar fuertes ví...",4
2667,"Prensa feminista, 2007), pág. 212. Comenzó a e...",5
1374,"El maestro de preescolar, el maestro de primer...",4
1106,Aquellos que hablan un idioma europeo tienen p...,4
4022,"Además, el plan no solo cubre al ejército cana...",5


La informacion cuenta con dos columnas, la primera Textos_espanol contiene el texto con la opinion del ciudadano, la segunda es el numero de ODS que le corresponde (columna que debemos predecir)

In [10]:
df_ods['sdg'].value_counts()

sdg
5    1451
4    1354
3    1244
Name: count, dtype: int64

# Preparacion de datos

Resolver problemas de codificacion

In [13]:
# Mapeo de caracteres UTF-8 mal interpretados a ANSI
utf8_to_ansi_map = {
    'Ã¡': 'á', 'Ã©': 'é', 'Ã­': 'í', 'Ã³': 'ó', 'Ãº': 'ú',
    'Ã±': 'ñ', 'ÃÁ': 'Á', 'Ã‰': 'É', 'ÃÍ': 'Í', 'ÃÓ': 'Ó',
    'ÃÚ': 'Ú', 'Ã‘': 'Ñ', 'Â¿': '¿', 'Â¡': '¡'
}


# Función de preprocesamiento de texto
def preprocess_text(text_series):
    spanish_stop_words = set(stopwords.words('spanish'))
    
    def replace_utf8_with_ansi(text):
        for utf8_char, ansi_char in utf8_to_ansi_map.items():
            text = text.replace(utf8_char, ansi_char)
        return text
    
    def clean_text(text):
        # Reemplazar caracteres UTF-8 mal interpretados
        #text = replace_utf8_with_ansi(text)
        # Convertir a minúsculas
        text = text.lower()
        # Tokenizar y eliminar puntuación y stopwords
        tokens = word_tokenize(text)
        tokens = [word for word in tokens if word.isalnum() and word not in spanish_stop_words]
        return ' '.join(tokens)

    # Aplicar el preprocesamiento completo
    return text_series.apply(clean_text)



Verificamos y todos los textos se encuentran en español por lo tanto es importante tener una buena preparcion de datos con respecto a los caracteres especiales.  Se define un mapeo entre los caracteres UTF-8 mal interpretados y sus equivalentes en ANSI para hacer este proceso. Se extrae una muestra de 5 filas aleatorias del DataFrame df_manipulado para verificar cómo quedaron los textos después de la corrección

Convertir en tokens

El objetivo es normalizar los textos para futuros análisis, eliminando caracteres irrelevantes y facilitando el procesamiento natural del lenguaje (NLP).

**Convertir a minúsculas (text.lower()):**

Esta conversión uniformiza las palabras, eliminando la diferencia entre mayúsculas y minúsculas. Por ejemplo, "Gobierno" y "gobierno" serán tratadas como la misma palabra.


**Tokenización (word_tokenize(text)):**

La tokenización separa el texto en palabras o "tokens". Esto para poder evaluar las palabras invidualmente
Ejemplo: "El consumo anual de alcohol" → ["El", "consumo", "anual", "de", "alcohol"].

**Eliminar puntuación y palabras vacías:**

Puntuación: Se eliminan tokens que no son alfanuméricos, como comas y puntos.
Stopwords: Las palabras vacías (palabras comunes como "de", "el", "y") se eliminan. Esto es útil porque estas palabras no suelen agregar significado a los análisis de texto.


**Unir los tokens (' '.join(tokens)):**

Se reconstruye el texto a partir de los tokens filtrados. El resultado es un texto sin puntuación, palabras comunes, y con solo palabras relevantes para el análisis.

Convertimos cada texto en un vector numérico, donde cada entrada representa la relevancia de una palabra para ese texto en particular. Este vector será usado como entrada para el modelo de clasificación.

# Construccion del modelo

In [19]:
preprocessor = FunctionTransformer(preprocess_text, validate=False)
# Crear el pipeline completo
pipeline2 = Pipeline([
    ('preprocessing', preprocessor), #('preprocessing', TextPreprocessor(utf8_to_ansi_map, stop_words=stopwords.words('spanish'))),
    ('tfidf', TfidfVectorizer()),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),
])

pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(
        lowercase=True,          # Convertir el texto a minúsculas automáticamente
        stop_words=stopwords.words('spanish'),    # Eliminar las stopwords en español (disponible en scikit-learn)
        #token_pattern=r'\b\w+\b' # Tokenización basada en palabras alfanuméricas
    )),
    ('rf', RandomForestClassifier(n_estimators=100, random_state=42)),  # Clasificador Random Forest
])

df_manipulado = df_ods.copy()

# Separar las variables
X = df_manipulado['Textos_espanol']  # Columna de texto
y = df_manipulado['sdg']  # Columna objetivo

## Random forest

In [21]:
# Dividimos los datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)


# Predicciones y evaluación
y_pred = pipeline.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

# Evaluamos el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=['ODS 3', 'ODS 4', 'ODS 5'], digits=4)
print(accuracy)
print(report)

0.9679012345679012
              precision    recall  f1-score   support

       ODS 3     0.9537    0.9880    0.9705       250
       ODS 4     0.9665    0.9701    0.9683       268
       ODS 5     0.9823    0.9486    0.9652       292

    accuracy                         0.9679       810
   macro avg     0.9675    0.9689    0.9680       810
weighted avg     0.9682    0.9679    0.9679       810



In [22]:
with open('random_forest_pipeline.pkl', 'wb') as file:
    pickle.dump(pipeline, file)

# Procesamiento de datos TEST

In [24]:
ubi_test = './data/TestODScat_345.xlsx'
df_test = pd.read_excel(ubi_test)

Resolver problemas de codificacion

In [26]:
df_test_manipulado = df_test.copy()

In [27]:
y_pred = pipeline.predict(df_test_manipulado['Textos_espanol'])
df_test["sdg"] = y_pred
df_test.sample(20)

,Textos_espanol,sdg
399,La propagación del VIH/SIDA invierte ese proce...,3
496,Si bien existen variaciones entre los países e...,5
244,"En realidad, los defensores de estos puntos de...",5
29,Algunos municipios también han introducido pol...,4
15,Incluso en los países de la OCDE considerados ...,3
495,"Sin embargo, se pueden ver signos alentadores ...",3
639,Las mujeres con solo escolaridad obligatoria t...,5
560,"Se informan a nivel de escuela, se comparan co...",4
568,"Las reformas laborales, la legislación para co...",5
6,"A diferencia de los acuerdos anteriores, la nu...",4


In [28]:
df_test["sdg"].value_counts()

sdg
4    265
5    250
3    187
Name: count, dtype: int64